# Student Loan Risk with Deep Learning

In [30]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [31]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [32]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [33]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [34]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [35]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking")

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [36]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [37]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [38]:
# Define the the number of inputs (features) to the model
number_of_features = X_train_scaled.shape[1]

# Review the number of features
number_of_features

11

In [39]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = (number_of_features + 1) // 2

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = hidden_nodes_layer1 // 2

# Define the number of neurons in the output layer
output_neurons = 1

In [40]:
# Create the Sequential model instance
nn = tf.keras.models.Sequential()

# Add the first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_of_features, activation="relu"))

# Add the second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [41]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 72        
                                                                 
 dense_4 (Dense)             (None, 3)                 21        
                                                                 
 dense_5 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [42]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [43]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
38/38 [==============================] - 1s 3ms/step - loss: 0.6908 - accuracy: 0.5346
Epoch 2/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6850 - accuracy: 0.6197
Epoch 3/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6799 - accuracy: 0.6622
Epoch 4/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6741 - accuracy: 0.7006
Epoch 5/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6671 - accuracy: 0.7139
Epoch 6/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6588 - accuracy: 0.7306
Epoch 7/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6491 - accuracy: 0.7306
Epoch 8/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6388 - accuracy: 0.7314
Epoch 9/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6284 - accuracy: 0.7373
Epoch 10/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6187 - accuracy: 0.7364
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [44]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5163 - accuracy: 0.7600 - 269ms/epoch - 21ms/step
Loss: 0.5163137316703796, Accuracy: 0.7599999904632568


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [45]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a HDF5 file
nn.save(file_path)


# Export your model to a keras file
nn.save("student_loans.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [46]:
# Set the model's file path
file_path = Path("student_loans.h5")

# Load the model
nn_imported = tf.keras.models.load_model(file_path)


# Load the model to a new object
nn_imported

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [47]:
# Make predictions with the test data
y_pred = nn_imported.predict(X_test_scaled)

# Display a sample of the predictions
y_pred[:5]

13/13 [==============================] - 0s 2ms/step


array([[0.3704835 ],
       [0.4329661 ],
       [0.74708194],
       [0.7432076 ],
       [0.96688735]], dtype=float32)

In [48]:
# Save the predictions to a DataFrame and round the predictions to binary results
results = pd.DataFrame(y_test)
results['predictions'] = y_pred
results['predictions'] = results['predictions'].apply(lambda x: 1 if x >= 0.5 else 0)
results.head()

,credit_ranking,predictions
75,0,0
1283,1,0
408,1,1
1281,1,1
1118,1,1


### Step 4: Display a classification report with the y test data and predictions

In [49]:
# Print the classification report with the y test data and predictions
print(classification_report(results["credit_ranking"], results["predictions"]))

              precision    recall  f1-score   support

           0       0.73      0.79      0.76       188
           1       0.80      0.74      0.76       212

    accuracy                           0.76       400
   macro avg       0.76      0.76      0.76       400
weighted avg       0.76      0.76      0.76       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.
  
  **To build a recommendation system for student loans, collect data on student finances, academic background, specific loan details, and historical repayment. This allows for personalized and viable loan recommendations.**

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.
  
  **Based on the data collected, the recommendation system would use content-based filtering. This approach matches loan options to individual students by leveraging detailed attributes of their financial and academic profiles, ensuring personalized and relevant recommendations.**

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.
  
  **Two real-world challenges in building a student loan recommendation system:**

**A. Privacy and Data Security: Handling sensitive financial and personal data requires robust security measures to protect against breaches and maintain user trust.**

**B. Bias and Fairness: Ensuring the system provides fair recommendations to all students, avoiding bias that could disadvantage certain groups.**

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

